# Student Tutorial - Spotify API

### Tutorial di Francesco Petrosino X81000533


![spotify.jpg](img/spotify.jpg)

<br>

## Cos'è Spotify

> **Spotify** è un servizio musicale svedese che offre lo *streaming on demand* di una selezione di brani di varie case discografiche ed etichette indipendenti, incluse *Sony, EMI, Warner Music Group e Universal*. Lanciato nell'ottobre 2008 dalla startup svedese Spotify AB, il servizio ha oltre 75 milioni di utenti al 10 giugno 2015. Il 14 dicembre 2016 vengono raggiunti i 40 milioni di utenti paganti, acquisendo in soli sei mesi 10 milioni di abbonati. Ha registrato più di 140 milioni di utenti attivi mensili nel giugno 2017 e più di 70 milioni di abbonati nel gennaio 2018.<br><br>
*Spotify* è disponibile in versione gratuita e a pagamento, nella maggior parte dei Paesi dell'Europa, nella quasi totalità delle Americhe, in Australia, in Nuova Zelanda e in alcuni Paesi dell'Asia. Il servizio è fruibile attraverso Microsoft Windows, macOS, GNU/Linux (solo per Debian e Ubuntu e senza supporto tecnico), Google Chrome OS, Telia Digital-tv e i dispositivi mobili equipaggiati con iOS (iPod/iPhone/iPad), Android, BlackBerry (in versione beta limitata), Windows Mobile, Windows Phone, S60 (Symbian), webOS, Squeezebox, Boxee, Sonos, Playstation 3, PlayStation 4, PlayStation 5, Xbox One, WD TV e MeeGo.<br><br>
La musica può essere visualizzata per artista, album, etichetta, genere o playlist così come grazie a ricerche dirette. Sui dispositivi desktop un collegamento permette all'ascoltatore di acquistare materiale selezionato presso venditori di terze parti.<br>

Da <a>https://it.wikipedia.org/wiki/Spotify</a>

## API di Spotify

Le API ufficiali fornite da Spotify permettono di ottenere informazioni sulle tracce per conoscere la loro valenza, numero di ascolti, numero di mi piace, e molto altro. Per casi d'uso più avanzati, è possibile ottenere dati di analisi approfonditi su tracce come: segmenti, tatums, bars, beats, pitches, e altro.<br>
I clients dovranno autenticarsi attraverso OAuth2.

![Spotify doc](img/spotify_doc.jpg)

Link alla documentazione: <a>https://developer.spotify.com/documentation/</a>

## Accesso alle API

Le API fornite da Spotify sono accessibili solamente attraverso un account Spotify Developer ed è quindi necessario sottoiscriversi a __[Spotify Developer Dashboard](https://developer.spotify.com/)__ per ottenere un token di autenticazione.

![Spotify developer](img/spotify_developer.JPG)

Per questo **Student Tutorial**, l'app si chiamerà *ProgettoSMM* (da Social Media Management) e sarà un piccolo script in Python per ottenere un semplice riepilogo dell'attività di un account Spotify e inoltre visualizzare informazioni sulle sue 30 tracks più ascoltate.<br>
Una volta cliccato su *create an app* la schermata si richiede alcune informazioni necessarie per poi restituirti una piccola dashboard personale.

![Spotify developer](img/spotify_developer_2.JPG)

Creata l'app, prendere nota di *Client ID* e *Client Secret*. Fatto ciò cliccare su **Edit Settings** ed aggiungere un *Redirect URIs* (ad es: http://localhost:8080). Cliccando su *Save* avremo tutto il necessario per procedere con il tutorial.

## API Wrapper: SPOTIPY

Il wrapper usato in questo tutorial è __[Python Spotify API Wrapper](https://spotipy.readthedocs.io/en/2.22.0/)__. Si tratta di un pacchetto per Python il cui scopo è fornire un metodo di accesso semplice alle API di Spotify. L'obiettivo di SPOTIPY è infatti fornire strumenti facili da usare e nel rispetto di tutte le regole d'accesso alle API.

### SPOTIPY Quick Start

Come spiegato nella guida __[Quick Start di SPOTIPY](https://spotipy.readthedocs.io/en/2.22.0/#installation)__ creiamo un'instanza per poter contattare i vari metodi esposti dalla libreria, tramite i seguenti comandi:

In [ ]:
pip install spotipy

Inseriamo dunque le nostre credenziali fornite nei punti precedenti (durante la fase di creazione della nostra nuova app)

In [1]:
SPOTIPY_CLIENT_ID='b48898f561ab4533b7396439e6610ff4'
SPOTIPY_CLIENT_SECRET='2c5bfc5f50b74f899e7ae755f14df67d'
REDIRECT_URI='http://localhost:8080'

Importiamo le librerie necessarie per interagire con spotipy

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

Creiamo una nuova istanza - autenticata - così da poterla riutilizzare per richiamare i vari metodi della libreria

In [3]:
scope = [
    'user-top-read'
]

auth_manager = SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=scope
)

sp = spotipy.Spotify(auth_manager=auth_manager)

È possibile verificare l'identità dell'utente con cui si è effettuato l'accesso attraverso il metodo *current_user()* restituendo in questo caso il nome dell'account

In [4]:
current_user = sp.current_user()

if current_user :
    print('Wellcome,', current_user['display_name'])

Wellcome, Francesco Petrosino


Avendo così configurato tutto, è finalmente possibile arrivare all'obiettivo del tutorial.

## Ottenere le 30 tracks più ascoltate e le informazioni più rilevanti

Questo piccolo progetto fornisce un esempio di uso delle API di Spotify, mostrando alcune delle funzionalità offerte. Lo scopo di questo script è, come illustrato precedentemente, mostrare un piccolo resoconto di un qualunque account utente sulla piattaforma e una lista contenente le tracks più ascoltate al momento e alcune informazioni rilevanti.

Prima di procedere però, importiamo la libreria **pandas** necessaria per la manipolazione dei dati.<br>
> **Pandas** è una libreria di alto livello che mette a disposizione diversi strumenti e strutture dati per l'analisi dei dati. In particolare, Pandas è molto utile per caricare, manipolare e visualizzare i dati in maniera veloce e conveniente prima di passare all'analisi vera e propria.

In [5]:
import pandas

Definendo un dataframe, potremmo ispezionare alcune delle informazioni rilevanti delle varie tracks..<br>
> Un **DataFrame** è una tabella di dati in cui ogni riga rappresenta un elemento "sotto esame" della lista, e ogni colonna invece rappresenta una particolare informazione di quell'elemento.

Per esempio, l'utente che abbiamo scelto per il tutorial ha queste informazioni:

In [6]:
user = {
    'display_name': current_user['display_name'],
    'followers_num': current_user['followers']['total'],
    'profile_link': current_user['href'],
    'id': current_user['id'],
    'images': current_user['images'],
    'type': current_user['type'],
    'uri': current_user['uri']
}

user_df = pandas.DataFrame(user, index={'current_user'})

user_df

,display_name,followers_num,profile_link,id,images,type,uri
current_user,Francesco Petrosino,11,https://api.spotify.com/v1/users/21ktslj6lkgx6...,21ktslj6lkgx6wswmfwu5cwuq,"{'height': None, 'url': 'https://platform-look...",user,spotify:user:21ktslj6lkgx6wswmfwu5cwuq


Segue il codice per ottenere le tracks ascoltate di frequente dall'utente corrente

In [7]:
tracks_take = 30

raw_result = sp.current_user_top_tracks(time_range='medium_term', limit=tracks_take)

Costruiamo adesso un dataframe a partire dal risultato visto in precedenza

In [8]:
tracks = []
columns_df = ['Id', 'Name', 'Artist name', 'Artist uri', 'Album name', 'Album release date', 'Album total tracks', 'Album uri', 'Disc number', 'Duration (ms)', 'Explicit', 'Href', 'Popularity', 'Uri']

for track in raw_result['items'] :
    tracks.append([
        track['id'],
        track['name'],
        track['artists'][0]['name'],
        track['artists'][0]['uri'],
        track['album']['name'],
        track['album']['release_date'],
        track['album']['total_tracks'],
        track['album']['uri'],
        track['disc_number'],
        track['duration_ms'],
        track['explicit'],
        track['href'],
        track['popularity'],
        track['uri']
    ])
    
tracks_df = pandas.DataFrame(tracks, columns = columns_df)

tracks_df

,Id,Name,Artist name,Artist uri,Album name,Album release date,Album total tracks,Album uri,Disc number,Duration (ms),Explicit,Href,Popularity,Uri
0,7hxHWCCAIIxFLCzvDgnQHX,"Lemonade (feat. Gunna, Don Toliver & NAV)",Internet Money,spotify:artist:6MPCFvOQv5cIGfw3jODMF0,B4 The Storm,2020-08-28,17,spotify:album:2vGU0DOcfsDee0euvhl1iZ,1,195428,True,https://api.spotify.com/v1/tracks/7hxHWCCAIIxF...,78,spotify:track:7hxHWCCAIIxFLCzvDgnQHX
1,0O6u0VJ46W86TxN9wgyqDj,I Like You (A Happier Song) (with Doja Cat),Post Malone,spotify:artist:246dkjvS1zLTtiykXe5h60,Twelve Carat Toothache,2022-06-03,14,spotify:album:3HHNR44YbP7XogMVwzbodx,1,192840,True,https://api.spotify.com/v1/tracks/0O6u0VJ46W86...,90,spotify:track:0O6u0VJ46W86TxN9wgyqDj
2,76OOmMnaTR0ofp52ZhUSyT,Staring At The Bottle,Jerome,spotify:artist:4xcDVatLFh6qlcm41er3LV,Staring At The Bottle,2021-09-17,1,spotify:album:4HWRzHxqJyXWdvN427p8VU,1,151368,False,https://api.spotify.com/v1/tracks/76OOmMnaTR0o...,64,spotify:track:76OOmMnaTR0ofp52ZhUSyT
3,4zN21mbAuaD0WqtmaTZZeP,Ferrari,James Hype,spotify:artist:43BxCL6t4c73BQnIJtry5v,Ferrari,2022-04-01,1,spotify:album:6moZ4sNThthUAwCklyuPY8,1,186661,False,https://api.spotify.com/v1/tracks/4zN21mbAuaD0...,88,spotify:track:4zN21mbAuaD0WqtmaTZZeP
4,2tIyoiGw4F82z2C8zb7eZB,TU MI HAI CAPITO (feat. Sfera Ebbasta),Madame,spotify:artist:1vgQksyJ0IVz8y9XerEOy3,MADAME,2021-06-03,18,spotify:album:7oF6ed3aSHU6aDD4MD3LIr,1,211498,False,https://api.spotify.com/v1/tracks/2tIyoiGw4F82...,62,spotify:track:2tIyoiGw4F82z2C8zb7eZB
5,3rmo8F54jFF8OgYsqTxm5d,Bad Habits,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,=,2021-10-29,14,spotify:album:32iAEBstCjauDhyKpGjTuq,1,230746,False,https://api.spotify.com/v1/tracks/3rmo8F54jFF8...,88,spotify:track:3rmo8F54jFF8OgYsqTxm5d
6,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),Justin Bieber,spotify:artist:1uNFoZAHBGtllmzznpCI3s,Justice,2021-03-19,16,spotify:album:5dGWwsZ9iB2Xc3UKR0gif2,1,198081,True,https://api.spotify.com/v1/tracks/4iJyoBOLtHqa...,85,spotify:track:4iJyoBOLtHqaGxP12qzhQI
7,50nfwKoDiSYg8zOCREWAm5,Shivers,Ed Sheeran,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,=,2021-10-29,14,spotify:album:32iAEBstCjauDhyKpGjTuq,1,207853,False,https://api.spotify.com/v1/tracks/50nfwKoDiSYg...,88,spotify:track:50nfwKoDiSYg8zOCREWAm5
8,6yaNNYWPjkWHbh1jADEyJO,KUMITE,Salmo,spotify:artist:3hBQ4zniNdQf1cqqo6hzuW,FLOP,2021-10-01,17,spotify:album:7DvK67C21i6go7olhjvLgT,1,170018,False,https://api.spotify.com/v1/tracks/6yaNNYWPjkWH...,67,spotify:track:6yaNNYWPjkWHbh1jADEyJO
9,6z7tGTD6yvyloEm6K2H8EX,LUNA PIENA (con IRAMA),Rkomi,spotify:artist:056KMTw6IztdQjBmFfVyO3,TAXI DRIVER,2021-11-30,15,spotify:album:4D04TN7Kw7Bq98kfDjUmgh,1,166983,True,https://api.spotify.com/v1/tracks/6z7tGTD6yvyl...,67,spotify:track:6z7tGTD6yvyloEm6K2H8EX


## Conclusione

Questo student tutorial si pone come piccola guida di introduzione allo sviluppo di applicazioni per l'analisi dati e di statistica, illustrando i passaggi per ottenere l'autenticazione e presentando in breve le API ufficiali.<br><br>
Per approfondimenti, visitare la documentazione ufficiale delle API di Spotify: <a>https://developer.spotify.com/documentation/</a>